#### Cemetery check pictures in SPA och söka i SPA
* [This notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20cemetery.ipynb) 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-11-17 01:04:25.667780


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  


In [11]:
# wd:Q26257009. Gamla Gävle Behravningsplats
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
# wd:Q26257009. Gamla Gävle Behravningsplats
# Q252312

queryGefle = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P119 wd:Q26257009. 
  minus {?item wdt:P4819 ?c }
  OPTIONAL { ?item wdt:P569 ?birthDate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """

queryNoPicNorra1890 = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P119 wd:Q252312. 
  minus {?item wdt:P4819 ?c }
  minus {?item wdt:P18 ?d }
  OPTIONAL { ?item wdt:P569 ?birthDate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
  FILTER ( YEAR(?birthDate) < 1880)
} order by ?itemLabel """

 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#SPAdf = get_sparql_dataframe(endpoint_url, query)
#SPAdf = get_sparql_dataframe(endpoint_url, queryNoPicNorra1890)
SPAdf = get_sparql_dataframe(endpoint_url, queryGefle)

SPAdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   search     75 non-null     object
 1   item       76 non-null     object
 2   itemLabel  76 non-null     object
 3   year       75 non-null     object
dtypes: object(4)
memory usage: 2.5+ KB


In [12]:
  SPAdf

,search,item,itemLabel,year
0,Abraham Nordström 1796,http://www.wikidata.org/entity/Q88637598,Abraham Nordström,1796
1,Abraham Wilhelm Nordström 1828,http://www.wikidata.org/entity/Q88653430,Abraham Wilhelm Nordström,1828
2,Aina Wallberg-Ewerlöf 1890,http://www.wikidata.org/entity/Q46400003,Aina Wallberg-Ewerlöf,1890
3,Anna Lovisa Larsson 1864,http://www.wikidata.org/entity/Q87106605,Anna Lovisa Larsson,1864
4,Anna Maria Forssén 1870,http://www.wikidata.org/entity/Q88770303,Anna Maria Forssén,1870
...,...,...,...,...
71,Stig Gavlén 1927,http://www.wikidata.org/entity/Q88456505,Stig Gavlén,1927
72,Thekla Daniella Engelmark 1847,http://www.wikidata.org/entity/Q88249448,Thekla Daniella Engelmark,1847
73,Theresia Charlotta Lönnerberg 1841,http://www.wikidata.org/entity/Q89006806,Theresia Charlotta Lönnerberg,1841
74,Tommy Sandlin 1944,http://www.wikidata.org/entity/Q344440,Tommy Sandlin,1944


In [13]:
SPAdetail = "https://portrattarkiv.se/details/"
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
 #           urlPicture = urlbasePic + id
 #           urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")



 Abraham Nordström 1796 http://www.wikidata.org/entity/Q88637598
		 17.947783 Sten   Leijonhufvud  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABZeNw 	
		 15.675295 Olof   Fåhræus  -  1796 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAABKvA 	
		 15.412925 Axel   Belfrage  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKjEg 	
		 15.383724 Bernhard   von Beskow  -  1796 https://portrattarkiv.se/details/5TJc-sPXaKAAAAAAAAAntA 	
		 15.249573 Olof   Fåhraeus  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABZLeQ 	

 Abraham Wilhelm Nordström 1828 http://www.wikidata.org/entity/Q88653430
		 23.229904 Abraham Viktor   Rydberg  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4vjQ 	
		 22.259556 Abraham   Rydberg  -  1828 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAQrQ 	
		 20.905848 Viktor   Rydberg  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACd0Cw 	
		 20.717686 Viktor   Rydberg  -  1828 https://portrattarkiv.se/details

		 18.96548 Oscar   Lindskog  -  1886 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAABicg 	
		 17.329826 Oscar   Lindskog  -  1886 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAABicw 	
		 15.025613 Catharina   Pihl  -  1886 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAABkCg 	
		 14.827401 Catharina   Pihl  -  1886 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABTblQ 	
		 14.422718 Thorfrid   Anners  -  1886 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA3RLg 	

 Carl Gabriel Albin Säfvenberg 1863 http://www.wikidata.org/entity/Q87166354
		 21.860126 Carl   Liljeholm  -  1863 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADhIw 	
		 21.835947 Carl   Liljeholm  -  1863 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLEqQ 	
		 19.64646 Carl   Liljeholm  -  1863 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAAA5YQ 	
		 19.451761 Gabriel   Olsson  -  1863 https://portrattarkiv.se/details/vfWYdYT6EoAAAAAAAAAP-A 	
		 19.331284 Carl   Clase  -  1863 https://portrat

		 17.787468 Erik   Eklöf  -  1872 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAABXPw 	
		 17.78622 Ester   Larsson  -  1872 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAAAXUw 	
		 16.953392 Ester   Hägerström  -  1872 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZ0lA 	
		 16.568937 Ester   Rising  -  1872 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKIHQ 	

 Fredrik Renárd 1824 http://www.wikidata.org/entity/Q88458132
		 18.578121 Karl   Danelius  -  1824 https://portrattarkiv.se/details/_P7S9YzQCjAAAAAAAAAqnA 	
		 18.503477 Karl   Danelius  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLXoA 	
		 18.2443 Gustaf   von Schoultz  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABMKYQ 	
		 17.958632 Carl   Kruuse-af-Verchou  -  1824 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAvDg 	
		 17.650429 Johan   Pettersson  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABL9Hw 	

 Fredrik Wilhelm Petre 1852 http://www.wikidata.org/entity

		 19.153936 Lars   Flygare  -  1806 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAAALqA 	
		 16.939663 Pehr   Lundberg  -  1806 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABAqg 	
		 15.876593 Johan   Grewell  -  1806 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLlXA 	
		 14.529438 Gude   Gedda  -  1806 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLhfg 	
		 14.422425 Christer   Mörner  -  1806 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAABK6A 	

 None http://www.wikidata.org/entity/Q88352402
400

 Maria Axelina Vikström 1874 http://www.wikidata.org/entity/Q98150875
		 21.046053 Axel   Vikström  -  1874 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABkhBg 	
		 20.074596 Edina   Olsson  -  1874 https://portrattarkiv.se/details/vfWYdYT6EoAAAAAAAAAMaQ 	
		 16.852207 Edina   Olsson  -  1874 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABqqYw 	
		 16.209167 Hermann   Petersson  -  1874 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA64UQ 	
		 16.117466 

		 18.838287 Selma   Gerle  -  1846 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAA_Og 	
		 17.31732 Maria   Olsén  -  1846 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAR-g 	
		 17.153248 Carl   Peterson-Tretow  -  1846 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABL8rg 	

 Sigrid Paues 1868 http://www.wikidata.org/entity/Q87101582
		 18.212025 Sigrid   Högberg  -  1868 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAAAGkg 	
		 17.893627 Sigrid   Sparre  -  1868 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAASKA 	
		 17.118206 Sigrid   Een  -  1868 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZutA 	
		 15.97094 Sigrid   Böttiger  -  1868 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA3qNw 	
		 14.954772 Sigrid   Högberg  -  1868 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACMwQ 	

 Sofia Kristina Aurora Rydvall 1852 http://www.wikidata.org/entity/Q87630991
		 18.35212 Hampus   Montgomery  -  1852 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAA